<a href="https://colab.research.google.com/github/kanikakaushik12/ML-PROJECT/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [2]:
! cp kaggle.json ~/.kaggle/


In [3]:
!  chmod 600 ~/.kaggle/kaggle.json

In [4]:
! kaggle competitions download -c jigsaw-multilingual-toxic-comment-classification

100% 1.08G/1.08G [00:15<00:00, 99.0MB/s]
100% 1.08G/1.08G [00:15<00:00, 74.0MB/s]


In [5]:
! unzip jigsaw-multilingual-toxic-comment-classification.zip

Archive:  jigsaw-multilingual-toxic-comment-classification.zip
  inflating: jigsaw-toxic-comment-train-processed-seqlen128.csv  
  inflating: jigsaw-toxic-comment-train.csv  
  inflating: jigsaw-unintended-bias-train-processed-seqlen128.csv  
  inflating: jigsaw-unintended-bias-train.csv  
  inflating: sample_submission.csv   
  inflating: test-processed-seqlen128.csv  
  inflating: test.csv                
  inflating: test_labels.csv         
  inflating: validation-processed-seqlen128.csv  
  inflating: validation.csv          


In [6]:
import os
import pandas as pd
import  tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff

from plotly import graph_objects as go
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers  import LSTM, GRU, SimpleRNN
from keras.layers import Dense, Activation, Dropout
from keras.layers import Embedding
from keras.preprocessing import sequence, text
from keras.utils import to_categorical
from keras.layers import BatchNormalization
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.callbacks import EarlyStopping

import nltk
import re

In [7]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
  print('Running on TPU', tpu.master())
except ValueError:
  tpu = None

if tpu:
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.iniitialized_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
  strategy = tf.distribute.get_strategy()

print("REPLICAS:", strategy.num_replicas_in_sync)



REPLICAS: 1


In [8]:
train = pd.read_csv('/content/jigsaw-toxic-comment-train.csv')
validation = pd.read_csv('/content/validation.csv')
test = pd.read_csv('/content/test.csv')


In [9]:
train.drop(['severe_toxic','obscene','threat','insult','identity_hate'],axis=1,inplace=True)


In [10]:
train = train.loc[:12000,:]
train.shape

(12001, 3)

In [11]:
train['comment_text'].apply(lambda x:len(str(x).split())).max()

1403

In [12]:
def roc_auc(predictions, target):

  fpr, tpr, thresholds = metrics.roc_curve(target, predictions)
  roc_auc = metrics.auc(fpr, tpr)
  return roc_auc

In [13]:
xtrain,xvalid,ytrain,yvalid= train_test_split(train.comment_text.values, train.toxic.values,stratify=train.toxic.values,random_state=42,test_size=0.2,shuffle=True)

In [14]:
token = text.Tokenizer(num_words=None)
max_len = 1500
token.fit_on_texts(list(xtrain)+list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)
xtrain_pad = sequence.pad_sequences(xtrain_seq,maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq,maxlen=max_len)
word_index = token.word_index


In [15]:
%%time
with strategy.scope():
  model= Sequential()
  model.add(Embedding(len(word_index)+1,300,input_length=max_len))
  model.add(SimpleRNN(100))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1500, 300)         13049100  
                                                                 
 simple_rnn (SimpleRNN)      (None, 100)               40100     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 13089301 (49.93 MB)
Trainable params: 13089301 (49.93 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
CPU times: user 470 ms, sys: 132 ms, total: 602 ms
Wall time: 752 ms


In [16]:
model.fit(xtrain_pad, ytrain, epochs=5, batch_size=64*strategy.num_replicas_in_sync)

Epoch 1/5
150/150 [==============================] - 562s 4s/step - loss: 0.3123 - accuracy: 0.8951
Epoch 2/5
150/150 [==============================] - 549s 4s/step - loss: 0.1054 - accuracy: 0.9609
Epoch 3/5
150/150 [==============================] - 547s 4s/step - loss: 0.0117 - accuracy: 0.9974
Epoch 4/5
150/150 [==============================] - 541s 4s/step - loss: 0.0018 - accuracy: 0.9999
Epoch 5/5
150/150 [==============================] - 545s 4s/step - loss: 6.3921e-04 - accuracy: 1.0000


In [17]:
scores = model.predict(xvalid_pad)
print("Auc: %.2f%%" % (roc_auc(scores,yvalid)))

76/76 [==============================] - 29s 376ms/step
Auc: 0.85%


In [18]:
scores_model =[]
scores_model.append({'Model': 'SimpleRNN','AUC_Score':roc_auc(scores,yvalid)})

In [19]:
xtrain_seq[:1]

[[664,
  65,
  7,
  19,
  2262,
  14102,
  5,
  2262,
  20439,
  6071,
  4,
  71,
  32,
  20440,
  6620,
  39,
  6,
  664,
  65,
  11,
  8,
  20441,
  1502,
  38,
  6072]]

In [20]:
import numpy as np
from tqdm import tqdm

# Correct file path
glove_file_path = r'C:\glove_file.csv'

# Initialize the embeddings index dictionary
embeddings_index = {}

# Open the GloVe file in read mode with UTF-8 encoding
try:
    with open(glove_file_path, 'r', encoding='utf-8') as f:
        for line in tqdm(f, desc="Reading GloVe file"):
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    print('Found %s word vectors.' % len(embeddings_index))
except FileNotFoundError:
    print("File not found. Please check the path.")



File not found. Please check the path.


In [21]:
embedding_matrix = np.zeros((len(word_index)+1,300))
for word,i in tqdm(word_index.items()):
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

100%|██████████| 43496/43496 [00:00<00:00, 1033002.35it/s]


In [22]:
%%time
with strategy.scope():
  model = Sequential()
  model.add(Embedding(len(word_index)+1,300,weights=[embedding_matrix],input_length=max_len,trainable=False))
  model.add(LSTM(100,dropout=0.3,recurrent_dropout=0.3))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1500, 300)         13049100  
                                                                 
 lstm (LSTM)                 (None, 100)               160400    
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 13209601 (50.39 MB)
Trainable params: 160501 (626.96 KB)
Non-trainable params: 13049100 (49.78 MB)
_________________________________________________________________
CPU times: user 413 ms, sys: 270 ms, total: 683 ms
Wall time: 603 ms


In [ ]:
model.fit(xtrain_pad,ytrain,epochs=5,batch_size=64*strategy.num_replicas_in_sync)

Epoch 1/5
150/150 [==============================] - 1709s 11s/step - loss: 0.3377 - accuracy: 0.9053
Epoch 2/5
150/150 [==============================] - 1688s 11s/step - loss: 0.3140 - accuracy: 0.9053
Epoch 3/5
150/150 [==============================] - 1797s 12s/step - loss: 0.3148 - accuracy: 0.9053
Epoch 4/5
 25/150 [====>.........................] - ETA: 26:11 - loss: 0.3261 - accuracy: 0.9000

In [ ]:
scores = model.predict(xvalid_pad)
print("AUC: %.2f%%" % (roc_auc(scores,yvalid)))

In [ ]:
scores_model.append({'Model': 'LSTM','AUC_Score':roc_auc(scores,yvalid)})

In [ ]:
%%time
with strategy.scope():
  model = Sequential()
  model.add(Embedding(len(word_index)+1,300,weights=[embedding_matrix],input_length=max_len,trainable=False))
  model.add(SpatialDropout1D(0.3))
  model.add(GRU(300))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(X_train, y_train, epochs=5, batch_size=64*strategy.num_replicas_in_sync)

In [ ]:
scores = model.predict(xvalid_pad)
print("AUC: %.2f%%" % (roc_auc(scores,yvalid)))

In [ ]:
scores_model.append({'Model':'GRU','AUC_Score':roc_auc(scores,yvalid)})

In [ ]:
scores_model

In [ ]:

%%time
with Strategy.scope():
  model = Sequential()
  model.add(Embedding(len(word_index)+1,300,weights=[embedding_matrix],input_length=max_len,trainable=False))
  model.add(Bidirectioanl(LSTM(300, dropout =0.3 ,recurrent_dropout=0.3)))
  model.add(Dense(1,actiivation='sigmoid'))
  model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(xtrain_pad , ytrain,epoch=5,batch_size=64*strategy.num_replicas_in_sync)

In [ ]:
scores = model.predict(xvalid_pad)
print("AUC: %.2f%%" % (roc_auc(scores,yvalid)))

In [ ]:
scores_model.append({'Model':'Bidirectional LSTM','AUC_Score':roc_auc(scores,yvalid)})

In [ ]:
result = pd.DataFrame(scores_model).sort_values(by='AUC_Score',ascending=False)
results.style.background_gradient(cmap='Purples')

In [ ]:
fig = go.Figure(go.Funnelarea(text =results.Model,values =result.AUC_Score,title={"position":"top center","text":"Funnel-Chart of sentiment Distribution"}))

fig.show()

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

import transformers
from transformers import BertWordPieceTokenizer
from tqdm.notebook import tqdm
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors

In [ ]:
import pandas as pd

train1 = pd.read_csv('/content/jigsaw-toxic-comment-train.csv')
valid = pd.read_csv('/content/validation.csv')
test = pd.read_csv('/content/test.csv')
sub = pd.read_csv('/content/sample_submission.csv')

In [ ]:
def fast_encode(texts, tokenizer, chunk_size=256, maxlen=512):
    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding(max_length=maxlen)
    all_ids = []

    for i in tqdm(range(0, len(texts), chunk_size)):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    return np.array(all_ids)

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE

# Configuration
EPOCHS = 3
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 192


In [ ]:
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
tokenizer.save_pretrained('.')
fast_tokenizer = BertWordPieceTokenizer('vocab.txt', lowercase=False)
fast_tokenizer

In [ ]:
x_x_train = fast_enode(train1.comment_text.astype(str), fast_tokenizer, maxlen=MAX_LEN)
x_valid = fast_encode(valid.comment_text.astype(str), fast_tokenizer, maxlen=MAX_LEN)
x_test = fast_encode(test.content.astype(str), fast_tokenizer, maxlen=MAX_LEN)
y_train = train1.toxic.values
y_valid = valid.toxic.values

In [ ]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)
valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)
test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)

In [ ]:
def build_model(transformer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(cls_token)

    model = Model(inputs=input_word_ids ,outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
%%time
with strategy.scope():
    transformer_layer = (
        transformers.TFDistilBertModel
        .from_pretrained('distilbert-base-multilingual-cased')
    )
    model = build_model(transformer_layer, max_len=MAX_LEN)
model.summary()

In [ ]:
n_steps = x_train.shape[0] // BATCH_SIZE
train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=EPOCHS
)

In [ ]:
n_steps = x_valid.shape[0] // BATCH_SIZE
train_history_2 = model.fit(
    valid_dataset.repeat(),
    steps_per_epoch=n_steps,
    epochs=EPOCHS*2
)

In [ ]:
sub['toxic'] = model.predict(test_dataset, verbose=1)
sub.to_csv('submission.csv', index=False)